In [12]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O cats_and_dogs_filtered.zip

--2020-08-25 00:45:53--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.240, 172.217.3.176, 172.217.14.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  28.4MB/s    in 2.3s    

2020-08-25 00:45:56 (28.4 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [13]:
!unzip -o -qq cats_and_dogs_filtered.zip

In [1]:
from tensorflow import keras
import os, sys
from matplotlib import pyplot

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np

In [2]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()


In [3]:
# run the test harness for evaluating a model
def run_test_harness(model):
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    # create data generators
    train_datagen = ImageDataGenerator(rescale=1.0/255.0,
        width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_it = train_datagen.flow_from_directory('cats_and_dogs_filtered/train/',
        class_mode='binary', batch_size=64, target_size=(64, 64))
    test_it = test_datagen.flow_from_directory('cats_and_dogs_filtered/validation/',
        class_mode='binary', batch_size=64, target_size=(64, 64))
    # Create EarlyStopping callback which stopps after 5 epochs of non-increasing accuracy
    early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=100, callbacks=[early])
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it))
    print('Test Accuracy > %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)


In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [15]:

# define cnn model
def load_model_1_block_cnn():
    # load model
    model = keras.models.load_model('baseline.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [17]:
model_1_cnn = load_model_1_block_cnn()
model_1_cnn.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8388736   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 8,390,657
Trainable params: 8,388,865
Non-trainable params: 1,792
________________________________________

In [18]:
run_test_harness(model_1_cnn)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - 5s 152ms/step - loss: 0.6847 - accuracy: 0.5355 - val_loss: 0.6650 - val_accuracy: 0.5720
Epoch 2/100
32/32 [==============================] - 5s 147ms/step - loss: 0.6678 - accuracy: 0.6040 - val_loss: 0.6429 - val_accuracy: 0.6000
Epoch 3/100
32/32 [==============================] - 5s 146ms/step - loss: 0.6498 - accuracy: 0.6295 - val_loss: 0.6267 - val_accuracy: 0.6280
Epoch 4/100
32/32 [==============================] - 5s 147ms/step - loss: 0.6362 - accuracy: 0.6480 - val_loss: 0.6220 - val_accuracy: 0.6360
Epoch 5/100
32/32 [==============================] - 5s 146ms/step - loss: 0.6302 - accuracy: 0.6485 - val_loss: 0.6047 - val_accuracy: 0.6580
Epoch 6/100
32/32 [==============================] - 5s 150ms/step - loss: 0.6211 - accuracy: 0.6630 - val_loss: 0.6263 - val_accuracy: 0.6200
Epoch 7/100
32/32 [==============================] - 5s 14

In [5]:
def load_model_2_blocks_cnn():
    # load model
    model = keras.models.load_model('2blocks-cnn.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
model_2_cnn = load_model_2_blocks_cnn()
model_2_cnn.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2_input (InputLayer)  [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)              

In [7]:
run_test_harness(model_2_cnn)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
32/32 [==============================] - 8s 252ms/step - loss: 0.6886 - accuracy: 0.5490 - val_loss: 0.6746 - val_accuracy: 0.6360
Epoch 2/100
32/32 [==============================] - 5s 158ms/step - loss: 0.6737 - accuracy: 0.6160 - val_loss: 0.6545 - val_accuracy: 0.6740
Epoch 3/100
32/32 [==============================] - 5s 154ms/step - loss: 0.6514 - accuracy: 0.6765 - val_loss: 0.6325 - val_accuracy: 0.7230
Epoch 4/100
32/32 [==============================] - 5s 151ms/step - loss: 0.6342 - accuracy: 0.6980 - val_loss: 0.6129 - val_accuracy: 0.7250
Epoch 5/100
32/32 [==============================] - 5s 152ms/step - loss: 0.6110 - accuracy: 0.7165 - val_loss: 0.5952 - val_accuracy: 0.7330
Epoch 6/100
32/32 [==============================] - 5s 151ms/step - loss: 0.5986 - accuracy: 0.7275 - val_loss: 0.5779 - val

In [19]:
def load_model_3_blocks_cnn():
    # load model
    model = keras.models.load_model('3blocks-cnn.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [20]:
model_3_cnn = load_model_3_blocks_cnn()
model_3_cnn.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)        

In [21]:
run_test_harness(model_3_cnn)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - 5s 154ms/step - loss: 0.6893 - accuracy: 0.5385 - val_loss: 0.6621 - val_accuracy: 0.6390
Epoch 2/100
32/32 [==============================] - 5s 152ms/step - loss: 0.6460 - accuracy: 0.6715 - val_loss: 0.6234 - val_accuracy: 0.6980
Epoch 3/100
32/32 [==============================] - 5s 151ms/step - loss: 0.6076 - accuracy: 0.7140 - val_loss: 0.5857 - val_accuracy: 0.7330
Epoch 4/100
32/32 [==============================] - 5s 150ms/step - loss: 0.5750 - accuracy: 0.7350 - val_loss: 0.5568 - val_accuracy: 0.7420
Epoch 5/100
32/32 [==============================] - 5s 150ms/step - loss: 0.5431 - accuracy: 0.7600 - val_loss: 0.5380 - val_accuracy: 0.7460
Epoch 6/100
32/32 [==============================] - 5s 150ms/step - loss: 0.5199 - accuracy: 0.7750 - val_loss: 0.5172 - val_accuracy: 0.7510
Epoch 7/100
32/32 [==============================] - 5s 15

In [22]:
def load_model_4_blocks_cnn():
    # load model
    model = keras.models.load_model('4blocks-cnn.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model_4_cnn = load_model_4_blocks_cnn()
model_4_cnn.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)       

In [43]:
run_test_harness(model_4_cnn)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - 9s 278ms/step - loss: 0.6581 - accuracy: 0.6160 - val_loss: 0.6136 - val_accuracy: 0.6980
Epoch 2/100
32/32 [==============================] - 9s 272ms/step - loss: 0.5874 - accuracy: 0.7400 - val_loss: 0.5505 - val_accuracy: 0.7500
Epoch 3/100
32/32 [==============================] - 9s 271ms/step - loss: 0.5321 - accuracy: 0.7770 - val_loss: 0.5091 - val_accuracy: 0.7700
Epoch 4/100
32/32 [==============================] - 9s 271ms/step - loss: 0.4967 - accuracy: 0.7850 - val_loss: 0.4800 - val_accuracy: 0.7860
Epoch 5/100
32/32 [==============================] - 9s 271ms/step - loss: 0.4610 - accuracy: 0.7970 - val_loss: 0.4611 - val_accuracy: 0.7800
Epoch 6/100
32/32 [==============================] - 9s 271ms/step - loss: 0.4481 - accuracy: 0.8045 - val_loss: 0.4509 - val_accuracy: 0.7860
Epoch 7/100
32/32 [==============================] - 9s 27

In [25]:
def load_model_4_blocks_cnn_50_epochs():
    # load model
    model = keras.models.load_model('4blocks-cnn-50-epochs.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [26]:
model_4_cnn_50_epochs = load_model_4_blocks_cnn_50_epochs()
model_4_cnn_50_epochs.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)       

In [27]:
run_test_harness(model_4_cnn_50_epochs)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - 5s 158ms/step - loss: 0.6827 - accuracy: 0.5605 - val_loss: 0.6378 - val_accuracy: 0.6320
Epoch 2/100
32/32 [==============================] - 5s 151ms/step - loss: 0.5938 - accuracy: 0.7225 - val_loss: 0.5618 - val_accuracy: 0.7430
Epoch 3/100
32/32 [==============================] - 5s 152ms/step - loss: 0.5355 - accuracy: 0.7660 - val_loss: 0.5124 - val_accuracy: 0.7750
Epoch 4/100
32/32 [==============================] - 5s 156ms/step - loss: 0.4904 - accuracy: 0.7945 - val_loss: 0.4846 - val_accuracy: 0.7810
Epoch 5/100
32/32 [==============================] - 5s 153ms/step - loss: 0.4623 - accuracy: 0.8125 - val_loss: 0.4616 - val_accuracy: 0.7810
Epoch 6/100
32/32 [==============================] - 5s 152ms/step - loss: 0.4422 - accuracy: 0.8015 - val_loss: 0.4488 - val_accuracy: 0.7840
Epoch 7/100
32/32 [==============================] - 5s 15

In [7]:
# define cnn model
def load_model_5_blocks_cnn():
    # load model
    model = keras.models.load_model('5blocks-cnn.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model_5_cnn = load_model_5_blocks_cnn()
model_5_cnn.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5_input (InputLayer)  [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 256)        

In [46]:
run_test_harness(model_5_cnn)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - 9s 282ms/step - loss: 0.6817 - accuracy: 0.5745 - val_loss: 0.6290 - val_accuracy: 0.6490
Epoch 2/100
32/32 [==============================] - 9s 275ms/step - loss: 0.5727 - accuracy: 0.7020 - val_loss: 0.5522 - val_accuracy: 0.7280
Epoch 3/100
32/32 [==============================] - 9s 275ms/step - loss: 0.5114 - accuracy: 0.7575 - val_loss: 0.5197 - val_accuracy: 0.7400
Epoch 4/100
32/32 [==============================] - 9s 275ms/step - loss: 0.4753 - accuracy: 0.7795 - val_loss: 0.4964 - val_accuracy: 0.7530
Epoch 5/100
32/32 [==============================] - 9s 276ms/step - loss: 0.4540 - accuracy: 0.7840 - val_loss: 0.4759 - val_accuracy: 0.7670
Epoch 6/100
32/32 [==============================] - 9s 275ms/step - loss: 0.4428 - accuracy: 0.7835 - val_loss: 0.4751 - val_accuracy: 0.7660
Epoch 7/100
32/32 [==============================] - 9s 27

In [9]:
# define cnn model
def load_model_5_blocks_cnn_dropout():
    # load model
    model = keras.models.load_model('5blocks-cnn-50epochs-dropout.h5')    
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-4].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
model_5_cnn_dropout = load_model_5_blocks_cnn()
model_5_cnn_dropout.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5_input (InputLayer)  [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 256)        

In [14]:
run_test_harness(model_5_cnn_dropout)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
32/32 [==============================] - 5s 161ms/step - loss: 0.6764 - accuracy: 0.5865 - val_loss: 0.5979 - val_accuracy: 0.6960
Epoch 2/100
32/32 [==============================] - 5s 163ms/step - loss: 0.5604 - accuracy: 0.7165 - val_loss: 0.5260 - val_accuracy: 0.7510
Epoch 3/100
32/32 [==============================] - 5s 162ms/step - loss: 0.5168 - accuracy: 0.7430 - val_loss: 0.5009 - val_accuracy: 0.7610
Epoch 4/100
32/32 [==============================] - 5s 156ms/step - loss: 0.4719 - accuracy: 0.7855 - val_loss: 0.4747 - val_accuracy: 0.7770
Epoch 5/100
32/32 [==============================] - 5s 155ms/step - loss: 0.4522 - accuracy: 0.8015 - val_loss: 0.4616 - val_accuracy: 0.7820
Epoch 6/100
32/32 [==============================] - 5s 156ms/step - loss: 0.4384 - accuracy: 0.8040 - val_loss: 0.4569 - val